In [1]:
# 1. Load the dataset
from datasets import load_dataset
ds = load_dataset("fancyzhx/amazon_polarity")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
# 2. Tokenization
from transformers import AutoTokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [3]:
def tokenize_function(example):
    return tokenizer(example["content"], padding="max_length", truncation=True)

tokenized_datasets = ds.map(tokenize_function, batched=True)

In [4]:
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [5]:
train_dataset = tokenized_datasets["train"]
test_dataset = tokenized_datasets["test"]

In [6]:
# 5. Load a model for sequence classification
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# 6. Evaluation metrics
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = torch.argmax(torch.tensor(logits), dim=1)
  precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="binary")
  acc = accuracy_score(labels, predictions)
  return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [8]:
import transformers
print(transformers.__version__)

4.52.3


In [16]:
# 8. Define training arguments
from transformers import TrainingArguments
training_args = TrainingArguments(
    do_eval=True,
    output_dir="./amazon_model",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [19]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].shuffle(seed=42).select(range(2000)),  # small subset
)

In [20]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: zahrasalehi232001 (zahrasalehi232001-alzahra-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


TrainOutput(global_step=375, training_loss=0.2659889933268229, metrics={'train_runtime': 994.9791, 'train_samples_per_second': 6.03, 'train_steps_per_second': 0.377, 'total_flos': 794804391936000.0, 'train_loss': 0.2659889933268229, 'epoch': 3.0})

# **RNN**

In [22]:
# 1. Prepare data
texts = ds["train"]["content"][:5000]
labels = ds["train"]["label"][:5000]

In [24]:
# 2. Tokenization and vocab
from collections import Counter
def tokenize(text):
    return text.lower().split()

vocab_counter = Counter()
for text in texts:
    vocab_counter.update(tokenize(text))

vocab = {"<PAD>": 0, "<UNK>": 1}
for i, word in enumerate(vocab_counter.keys(), start=2):
    vocab[word] = i

def encode(text, max_len=256):
    tokens = tokenize(text)
    ids = [vocab.get(t, 1) for t in tokens[:max_len]]
    return ids + [0] * (max_len - len(ids))

In [29]:
# 3. Dataset class
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.inputs = [torch.tensor(encode(t)) for t in texts]
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.inputs[idx], self.labels[idx]

rnn_dataset = TextDataset(texts, labels)
rnn_loader = DataLoader(rnn_dataset, batch_size=32, shuffle=True)

In [30]:
# 4. RNN model

class SentimentRNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 2)

    def forward(self, x):
        x = self.embedding(x)
        _, (h_n, _) = self.lstm(x)
        return self.fc(h_n[-1])

model_rnn = SentimentRNN(len(vocab), 128, 128)

In [31]:
# 5. Training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_rnn.to(device)
optimizer = torch.optim.Adam(model_rnn.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

for epoch in range(3):
    model_rnn.train()
    total_loss = 0
    for x_batch, y_batch in rnn_loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model_rnn(x_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}: Loss = {total_loss / len(rnn_loader):.4f}")

Epoch 1: Loss = 0.6912
Epoch 2: Loss = 0.6908
Epoch 3: Loss = 0.6907


# **transformer**

In [32]:
# Positional Encoding
class PositionalEncoding(nn.Module):
    def __init__(self, dim_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, dim_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, dim_model, 2) * (-math.log(10000.0) / dim_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [33]:
# Transformer Encoder Block
class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, heads, ff_hidden, dropout):
        super().__init__()
        self.attention = nn.MultiheadAttention(embed_dim, heads, dropout=dropout, batch_first=True)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.ff = nn.Sequential(
            nn.Linear(embed_dim, ff_hidden),
            nn.ReLU(),
            nn.Linear(ff_hidden, embed_dim)
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        attn_output, _ = self.attention(x, x, x)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.ff(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [34]:
# Transformer-based Classifier
class TransformerClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, ff_hidden, num_layers, num_classes, max_len=256):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.pos_encoding = PositionalEncoding(embed_dim, max_len)
        self.layers = nn.ModuleList([
            TransformerBlock(embed_dim, num_heads, ff_hidden, dropout=0.1) for _ in range(num_layers)
        ])
        self.classifier = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = self.pos_encoding(x)
        for layer in self.layers:
            x = layer(x)
        x = x[:, 0]  # first token summary (like [CLS])
        return self.classifier(x)

In [36]:
# Instantiate model (example configuration)
import math
TransformerClassifier(
    vocab_size=10000,  # replace with actual vocab size
    embed_dim=128,
    num_heads=4,
    ff_hidden=256,
    num_layers=2,
    num_classes=2
)

TransformerClassifier(
  (embedding): Embedding(10000, 128, padding_idx=0)
  (pos_encoding): PositionalEncoding()
  (layers): ModuleList(
    (0-1): 2 x TransformerBlock(
      (attention): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
      )
      (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (ff): Sequential(
        (0): Linear(in_features=128, out_features=256, bias=True)
        (1): ReLU()
        (2): Linear(in_features=256, out_features=128, bias=True)
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (classifier): Linear(in_features=128, out_features=2, bias=True)
)